### <font color='#4285f4'>Overview</font>

Maximize your advertising ROI with cutting-edge campaign optimization forecasting. By analyzing your ad creative and historical data, we explore the highest-performing channels and predict the ideal budget split for maximum profit using Gemini and TimesFM forecasting.

Process Flow:

1. Download the marketing videos created by GenAI
2. Download the YouTube short videos
3. Provide Gemini a list of Marketing Channels we use
4. Prompt Gemini to provide us the best marketing channels and how to allocate our budget by watching both ads.
5. Perform 3 different TimesFM scenarios
    * a. Scenario 1 - We spend $2000 on each campaign individually (run each campaign one by one)
    * b. Scenario 2 - We spend $3000 on autonomous_taxi and youtube_shorts individually
    * c. Scenario 3 - We spend $2000 on all 3 campaigns at the same time
6. Run all the TimesFM simulations
7. Output which simulation has the highest profit
8. Show the output as charts

Notes:

* You may run as many different scenarios as you would like.  This notebook does not run every possible scenario and is designed to get you started with TimesFM.

Cost:
* Low: Gemini, BigQuery, TimesFM (locally)
* Medium: Remember to stop your Colab Enterprise Notebook Runtime

Author:
* Adam Paternostro

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Artifacts/Create-Campaign-Target-Channels-Architecture.png', width=1200)

### <font color='#4285f4'>Video Walkthrough</font>

[![Video](https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/adam-paternostro-video.png)](https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Create-Campaign-Target-Channels.mp4)


In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Create-Campaign-Target-Channels.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>License</font>

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

### <font color='#4285f4'>Deploy TimesFM</font>

1. Open Vertex Model Garden
   -  https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/timesfm
2. Click the Deploy button
3. Select
   - Resource Id:  google/timesfm-v20240828
   - Model Name: (leave default - name does not matter)   
   - Endpoint name: (leave default - name does not matter)
   - Region: us-central1 (if you change you need to change the **Initialize** variables below)
   - Machine spec: (leave default - n1-standard-8)
4. Click Deploy
5. Wait 20 minutes
6. Open Vertex Model Registry
   - https://console.cloud.google.com/vertex-ai/models
7. Click on the model name
8. Click on the model name under "Deploy your model"
9. Click on "Sample Request" (at the top)
10. Copy the ```ENDPOINT_ID="000000000000000000"```
11. Update the variable endpoint_id in the **Initialize** code below.

##### TimesFM Deployment Video

[![TimesFM Deployment Video](../images/adam-paternostro-video.png)](https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Campaign-Performance-Forecasting-TimesFM-Install.mp4)

In [ ]:
from IPython.display import HTML

HTML("""
<h2>Deploying TimesFM to a Vertex AI Endpoint Instructions</h2>
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/chocolate-ai/v1/Videos/Campaign-Performance-Forecasting-TimesFM-Install.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>Pip installs</font>

In [ ]:
# PIP Installs
import sys

# https://PLACEHOLDER.com/index.html

# For better performance and production, deploy to Vertex AI endpoint with GPU
# This takes about 5 minutes to install and you will need to reset your runtime
# !{sys.executable} -m pip install timesfm <- THERE ARE TOO MANY DEPENDENCIES TO EASILY TO THIS IN COLAB

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
from IPython.display import Audio
from functools import reduce
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

endpoint_id="000000000000000000"  # <- YOU MUST SET THIS !!!!

bigquery_location = "${bigquery_location}"
region = "${region}"
location = "${location}"
storage_account = "${chocolate_ai_bucket}"
public_storage_storage_account = "data-analytics-golden-demo"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Helper Methods</font>

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### RetryCondition (for retrying LLM calls)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### timesFMInference
Calls TimesFM Vertex Model Endpoint

In [ ]:
def timesFMInference(project_number, endpoint_id, payload):
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_number}/locations/{location}/endpoints/{endpoint_id}:predict"
  # print(f"url: {url}")
  response = restAPIHelper(url, http_verb="POST", request_body=payload)
  # print(f"response: {response}")
  return response

#### getProjectNumber
Gets the project number from a project id

In [ ]:
def getProjectNumber(project_id):
  """Batch activates service apis"""

  url = f"https://cloudresourcemanager.googleapis.com/v1/projects/{project_id}"
  json_result = restAPIHelper(url, "GET", None)
  print(f"getProjectNumber (GET) json_result: {json_result}")

  project_number = json_result["projectNumber"]
  return project_number

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-2.0-flash", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-2.0-flash"

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-2.0-flash":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-2.0-flash", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-2.0-flash"

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-2.0-flash":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Helper Functions

In [ ]:
# prompt: python to delete a file even if it does not exist

def delete_file(filename):
  try:
    os.remove(filename)
    print(f"File '{filename}' deleted successfully.")
  except FileNotFoundError:
    print(f"File '{filename}' not found.")

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json_formatted_str

In [ ]:
# prompt: python code to download a pdf from the internet

import requests

def download_http_file(url, filename):
  """Downloads a PDF file from a given URL.

  Args:
      url: The URL of the PDF file to download.
      filename: The name to save the downloaded PDF file as.
  """
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(filename, 'wb') as f:
      f.write(response.content)

    print(f"PDF downloaded successfully to {filename}")

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while downloading the PDF: {e}")

# Example usage:


In [ ]:
def download_from_gcs(filename, gcs_storage_bucket, gcs_storage_path):
  # prompt: Write python code to download a blob from a gcs bucket.  do not use the requests method

  from google.cloud import storage

  # The ID of your GCS object
  object_name = gcs_storage_path + filename

  # The path to which the file should be downloaded
  destination_file_name = filename

  storage_client = storage.Client()

  bucket = storage_client.bucket(gcs_storage_bucket)

  # Construct a client side representation of a blob.
  # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
  # any content from Google Cloud Storage. As we don't need additional data,
  # using `Bucket.blob` is preferred here.
  blob = bucket.blob(object_name)
  blob.download_to_filename(destination_file_name)

  print(
      "Downloaded storage object {} from bucket {} to local file {}.".format(
          object_name, gcs_storage_bucket, destination_file_name
      )
  )

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

### <font color='#4285f4'>Target Channels</font>

In [ ]:
# Download our sample Chocolate A.I. Ad and upload to our project storage account

print(f"To browse the public storage account: https://console.cloud.google.com/storage/browser/{public_storage_storage_account}/chocolate-ai/v1")
print("")

# Full videos
download_from_gcs("full-video-with-audio-en-GB.mp4", "data-analytics-golden-demo", "chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-01/")
copy_file_to_gcs("full-video-with-audio-en-GB.mp4", storage_account, f"chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/story-01-full-video-with-audio-en-GB.mp4")

download_from_gcs("full-video-with-audio-en-GB.mp4", "data-analytics-golden-demo", "chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-02/")
copy_file_to_gcs("full-video-with-audio-en-GB.mp4", storage_account, f"chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/story-02-full-video-with-audio-en-GB.mp4")

download_from_gcs("full-video-with-audio-en-GB.mp4", "data-analytics-golden-demo", "chocolate-ai/v1/Campaign-Assets-Text-to-Video-01/story-03/")
copy_file_to_gcs("full-video-with-audio-en-GB.mp4", storage_account, f"chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/story-03-full-video-with-audio-en-GB.mp4")

# YouTube Shorts
download_from_gcs("youtube-short-extract-clip-story-01-final.mp4", "data-analytics-golden-demo", "chocolate-ai/v1/Campaign-Assets-Video-Create-Shorts/")
copy_file_to_gcs("youtube-short-extract-clip-story-01-final.mp4", storage_account, f"chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/youtube-short-extract-clip-story-01-final.mp4")

download_from_gcs("youtube-short-extract-clip-story-02-final.mp4", "data-analytics-golden-demo", "chocolate-ai/v1/Campaign-Assets-Video-Create-Shorts/")
copy_file_to_gcs("youtube-short-extract-clip-story-02-final.mp4", storage_account, f"chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/youtube-short-extract-clip-story-02-final.mp4")

download_from_gcs("youtube-short-extract-clip-story-03-final.mp4", "data-analytics-golden-demo", "chocolate-ai/v1/Campaign-Assets-Video-Create-Shorts/")
copy_file_to_gcs("youtube-short-extract-clip-story-03-final.mp4", storage_account, f"chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/youtube-short-extract-clip-story-03-final.mp4")

print("")
print(f"To view in your project storage account: https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}")


In [ ]:
# Prompt: You work at a marketing company and need a list of advertising channels. Create a list channels along with a description for each.
# Output as a Python Dictionary
# Include the following fields: ...

advertising_channels = {
    "Digital Advertising": {
        "Search Engine Marketing (SEM)": {
            "description": "Paid advertising on search engine results pages (SERPs) like Google, Bing, etc.",
            "vendors": ["Google Ads", "Bing Ads"],
            "pros": ["Highly targeted", "Measurable", "Quick results"],
            "cons": ["Can be expensive", "Requires ongoing management"]
        },
        "Search Engine Optimization (SEO)": {
            "description": "Optimizing your website to rank higher organically in search engine results.",
            "vendors": ["SEMrush", "Ahrefs", "Moz" ,"Google Search Console"], # Tools rather than vendors
            "pros": ["Long-term ROI", "Increased credibility and brand authority"],
            "cons": ["Requires time and effort", "Results are not immediate"]
        },
        "Social Media Advertising": {
            "description": "Paid ad campaigns on platforms like Facebook, Instagram, Twitter, LinkedIn, etc.",
            "vendors": ["YouTube Shorts", "Facebook Ads", "Instagram Ads", "Twitter Ads", "LinkedIn Ads", "TikTok Ads", "Pinterest Ads"],
            "pros": ["Precise audience targeting", "Diverse ad formats", "Relatively affordable"],
            "cons": ["Can be perceived as intrusive", "Requires platform expertise"]
        },
        "Autonomous Taxi Advertising": {
            "description": "Show videos to customers that are near a specific geospatial coordinate.",
            "vendors": ["Autonomous Taxi"],
            "pros": [
                "Hyperlocal Targeting", # Reaching customers precisely where they are
                "Captive Audience",  # Passengers have limited distractions during rides
                "Measurable Results", # Track ad views, engagement, and even potential store visits
                "Novelty Factor", #  New and potentially engaging ad format
                "Integration with Ride Experience", # Ads can be tailored to destinations or journey times
            ],
            "cons": [
                "Privacy Concerns", # Targeting based on location data requires transparency and user consent
                "Limited Reach",  #  Reaches only those using Autonomous Taxi services
                "Potential for Intrusiveness", #  Striking a balance between advertising and passenger experience is crucial
                "Technical Challenges", #  Requires robust geofencing and ad delivery technology
                "Cost Considerations", #  Pricing models and potential ROI need to be established
            ]
        },
        "Display Advertising": {
            "description": "Visual ads on websites, apps, and social media platforms, often using banner ads or rich media.",
            "vendors": ["Google Display Network", "DoubleClick", "Adform", "The Trade Desk"],
            "pros": ["Broad reach", "Visual impact", "Various targeting options"],
            "cons": ["Can be ignored", "Ad blindness is a factor", "Can be expensive"]
        },
        "Video Advertising": {
            "description": "Ads displayed on platforms like YouTube, Vimeo, streaming services, and social media.",
            "vendors": ["YouTube Ads", "Vimeo Ads", "Amazon Advertising", "Hulu Ads"],
            "pros": ["Engaging format", "High completion rates", "Strong storytelling potential"],
            "cons": ["Higher production costs", "Ad skipping can be an issue"]
        },
        "Mobile Advertising": {
            "description": "Ads specifically targeted to mobile devices, through apps, mobile websites, and SMS.",
            "vendors": ["Google Mobile Ads", "Apple Search Ads", "InMobi", "AdColony"],
            "pros": ["Reaches a vast and engaged audience", "Highly personalizable"],
            "cons": ["Can be intrusive", "Ad fatigue is common", "Requires mobile optimization"]
        },
        "Email Marketing": {
            "description": "Sending targeted marketing messages directly to an audience's inbox.",
            "vendors": ["Mailchimp", "Campaign Monitor", "Constant Contact", "GetResponse"],
            "pros": ["Direct communication", "High ROI", "Trackable results"],
            "cons": ["Requires list building", "Can be flagged as spam", "Needs compelling content"]
        },
        "Affiliate Marketing": {
            "description": "Partnering with other businesses or individuals to promote your products or services for a commission.",
            "vendors": ["Amazon Associates", "ShareASale", "CJ Affiliate", "ClickBank"],
            "pros": ["Performance-based", "Access to new audiences", "Cost-effective"],
            "cons": ["Requires relationship building", "Can be complex to manage"]
        },
        "Influencer Marketing": {
            "description": "Collaborating with influential people on social media or other platforms to promote your brand.",
            "vendors": ["Influencer marketing platforms", "Talent agencies", "Direct outreach to influencers"], # Less specific vendors, more about approach
            "pros": ["Reaches a targeted audience", "Builds credibility and trust"],
            "cons": ["Can be expensive", "Requires careful vetting of influencers"]
        },
        "Native Advertising": {
            "description": "Ads that blend in with the surrounding content on a platform, aiming to be less intrusive.",
            "vendors": ["Outbrain", "Taboola", "Yahoo Gemini", "TripleLift"],
            "pros": ["High engagement rates", "Less ad fatigue", "Perceived as valuable content"],
            "cons": ["Can be deceptive if not executed ethically", "Requires careful planning"]
        }
    },
    "Traditional Advertising": {
        "Television Advertising": {
            "description": "Commercials aired during television programs.",
            "vendors": ["ITV", "Channel 4", "Sky", "London TV"],
            "pros": ["Broad reach", "High impact", "Strong emotional connection"],
            "cons": ["Expensive", "Declining viewership", "Difficult to target specific audiences"]
        },
        "Radio Advertising": {
            "description": "Ads broadcasted on radio stations.",
            "vendors": ["Global", "Bauer Media", "Wireless Group", "Communicorp"], # Major radio groups in London
            "pros": ["Local targeting", "Affordable", "Can reach people on the go"],
            "cons": ["Audio-only format", "Can be easily tuned out", "Difficult to track ROI"]
        },
        "Print Advertising": {
            "description": "Ads placed in newspapers, magazines, brochures, etc.",
            "vendors": ["Evening Standard", "Metro", "Time Out London", "The Londonist"], # Popular London publications
            "pros": ["Tangible format", "Targeted reach for specific audiences", "Longer shelf life"],
            "cons": ["Declining readership", "Long lead times", "Difficult to track effectiveness"]
        },
        "Out-of-Home Advertising (OOH)": {
            "description": "Ads placed in public spaces, such as billboards, transit ads, street furniture, etc.",
            "vendors": ["JCDecaux", "Global", "Clear Channel", "ES Digital"], # Major OOH companies in London
            "pros": ["High visibility", "Constant exposure", "Geographically targeted"],
            "cons": ["Difficult to measure effectiveness", "Limited messaging space", "Can be costly"]
        },
        "Direct Mail Marketing": {
            "description": "Sending physical marketing materials, like postcards, flyers, catalogs, directly to potential customers.",
            "vendors": ["Royal Mail MarketReach", "Printing companies", "Mailing houses"], # More about service providers
            "pros": ["Tangible and personalized", "Can be highly targeted"],
            "cons": ["High production and postage costs", "Often perceived as junk mail"]
        }
    },
    "Emerging Advertising Channels": {
        "Augmented Reality (AR) & Virtual Reality (VR) Advertising": {
            "description": "Interactive and immersive ad experiences.",
            "vendors": ["Snapchat", "Facebook (Spark AR)", "Zappar", "Blippar"] # Platforms and AR development companies
        },
        "Connected TV (CTV) Advertising": {
            "description": "Targeted ads on streaming services accessed through internet-connected TVs.",
            "vendors": ["Hulu", "Amazon Advertising", "Roku", "Samsung Ads"]
        },
        "Digital Audio Advertising": {
            "description": "Ads on music streaming platforms, podcasts, and digital radio.",
            "vendors": ["Spotify", "Apple Music", "Pandora", "Acast"],
        },
        "Programmatic Advertising": {
            "description": "Using technology to automate the buying and selling of ad space in real-time.",
            "vendors": ["Google Display & Video 360", "The Trade Desk", "Adobe Advertising Cloud", "Amazon DSP"]
        }
    }
}

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  [{
#    "channel" : "text",
#    "channel-video": "text",
#    "channel-budget": 1.5,
#    "channel-vendors" : ["text"],
#    "channel-ranking" : 1,
#    "channel-pros" : ["text"],
#    "channel-cons" : ["text"],
#    "explanation" : "text"
#  }]
response_schema = {
  "type": "array",
  "items": {
    "type": "object",
    "required": [
      "channel",
      "channel-video",
      "channel-budget",
      "channel-vendors",
      "channel-ranking",
      "channel-pros",
      "channel-cons",
      "explanation"
    ],
    "properties": {
      "channel": {
        "type": "string"
      },
      "channel-video": {
        "type": "string"
      },
      "channel-budget": {
        "type": "number"
      },
      "channel-vendors": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "channel-ranking": {
        "type": "integer"
      },
      "channel-pros": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "channel-cons": {
        "type": "array",
        "items": {
          "type": "string"
        }
      },
      "explanation": {
        "type": "string"
      }
    }
  }
}

prompt = f"""You are an expert at creating marketing campaigns.
You work at Chocolate A.I. a company that sells chocolates, desserts and coffee based in Paris, France.
I am providing you two videos, the full marketing campaign and a short video.
I need you to recommend the top 3 channels to use for the new ad.
I have a $4,000 budget, allocate the budget to the top 3 channels.
You can use the full ad or the short.
Make sure you explain why in the explaination field.

Output the following JSON in OpenAPI 3.0 schema object:
channel: the name of the channel
channel-video: the name of video that the channel is targeted for
channel-vendors: the names of the vendors that the channel is targeted for
channel-ranking: the ranking of the channel.  Rank the channels starting at 1.  1 is the best channel.
channel-pros: the pros of the channel
channel-cons: the cons of the channel
explanation: a brief explanation of the channel

<AdvertisingChannels>
{advertising_channels}
</AdvertisingChannels>
"""

story_number = "01" # You can also do 02 or 03 for the other two videos

multimodal_prompt_list = [
    { "text": prompt },
    { "fileData": {  "mimeType": "video/mp4", "fileUri": f"gs://{storage_account}/chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/story-{story_number}-full-video-with-audio-en-GB.mp4" } },
    { "fileData": {  "mimeType": "video/mp4", "fileUri": f"gs://{storage_account}/chocolate-ai/Create-Campaign-Target-Channels/text-to-videos-{formatted_date}/youtube-short-extract-clip-story-{story_number}-final.mp4" } },
  ]

channel_response = GeminiLLM_Multimodal(multimodal_prompt_list, response_schema=response_schema)

In [ ]:
result = PrettyPrintJson(channel_response)
print(result)

### <font color='#4285f4'>TimesFM</font>

In [ ]:
# import timesfm

In [ ]:
context_len = 512
horizon_len = 128 # Predict next 128 days, this could be 128 without requiring compute (129 would be a step up).  This is more of the max horizon len.
input_patch_len = 32
output_patch_len = 128
num_layers = 20
model_dims = 1280
timesfm_backend = "cpu" # cpu, gpu or cuda
xreg_mode = "xreg + timesfm"

# from jax._src import config
# config.update("jax_platforms", timesfm_backend)

# model = timesfm.TimesFm(
#     context_len=512,
#     horizon_len=128, # Predict next {x} days, this could be 128 without requiring compute (129 would be a step up).  This is more of the max horizon len.
#     input_patch_len=32,
#     output_patch_len=128,
#     num_layers=20,
#     model_dims=1280,
#     backend=timesfm_backend,
# )

# Load the model
# This can produce "ERROR:absl:For checkpoint version > 1.0, we require users to provide", you can ignore that
# model.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

### <font color='#4285f4'>Predict Campaign Sales</font>


#### Our source data for TimesFM

In [ ]:
# These are from a spreadsheet that has for each campaign type the increase in sales
# We will use TimesFM to find the patterns
# We have ads that run indvidually, together and also adding more money does not necessary increase our returns (they plateau or the profit margin decreases)

autonomous_taxi_cumulative_spend = [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,400,600,800,1000,1200,1400,1600,1800,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,400,600,800,1000,1200,1400,1600,1800,2000,0,0,500,1000,1500,2000,0,0,0,0,0,0,0,0,0,0,0,500,1000,1500,2000,0,0,0,0,0,0,0,500,1000,1500,2000,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,0,0,0,0,0]
autonomous_taxi_campaign_day = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","","","1","2","3","4","","","","","","","","","","","","1","2","3","4","","","","","","","","1","2","3","4","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","","","","",""]
youtube_short_cumulative_spend = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,400,600,800,1000,1200,1400,1600,1800,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,400,600,800,1000,1200,1400,1600,1800,2000,0,0,0,0,0,0,0,500,1000,1500,2000,0,0,0,0,0,0,500,1000,1500,2000,0,0,500,1000,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,0,0,0,0,0]
youtube_short_campaign_day = ["","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","","","","","","","","1","2","3","4","","","","","","","1","2","3","4","","","1","2","3","4","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","","","","",""]
google_mobile_ads_cumulative_spend = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,400,600,800,1000,1200,1400,1600,1800,2000,0,0,200,400,600,800,1000,1200,1400,1600,1800,2000,0,0,0,0,0,0,0,0,0,0,0,0,500,1000,1500,2000,0,0,0,0,0,0,0,500,1000,1500,2000,0,500,1000,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
google_mobile_ads_campaign_day = ["","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","","","1","2","3","4","5","6","7","8","9","10","","","","","","","","","","","","","1","2","3","4","","","","","","","","1","2","3","4","","1","2","3","4","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","","","","","","","","","","","","","","","","","","","","","","","","","","","","","",""]

sales_increase = [69.00,103.60,138.41,173.02,208.66,244.41,279.88,314.87,632.89,705.32,777.40,709.56,569.92,429.57,287.81,144.48,130.16,116.28,102.25,102.25,0.00,0.00,29.48,73.70,118.27,163.27,208.01,252.58,297.75,343.44,388.23,433.46,480.70,526.81,572.54,453.81,424.83,365.60,304.97,229.18,153.55,76.78,0.00,0.00,154.48,154.63,232.64,233.34,233.34,311.43,344.28,329.95,314.24,251.64,221.07,189.68,158.06,126.96,95.22,63.48,31.80,15.93,15.98,16.00,0.00,0.00,113.10,202.37,292.58,383.89,475.16,567.53,658.67,749.24,1171.47,1307.38,1442.66,1415.13,1303.78,1005.92,807.15,572.30,488.14,388.31,288.44,203.61,0.00,0.00,257.32,300.51,429.29,473.17,516.70,648.46,727.00,756.73,1135.27,1153.89,1209.95,1087.20,876.78,667.68,458.66,247.96,195.22,160.20,142.97,143.54,0.00,0.00,215.96,270.49,416.00,472.61,529.25,676.60,768.03,808.62,847.92,831.97,853.86,875.91,896.34,709.60,638.08,526.00,413.70,301.78,207.47,113.51,0.00,0.00,142.45,237.42,333.38,429.06,958.24,958.24,576.09,192.61,154.70,136.04,0.00,0.00,98.34,216.57,335.37,455.10,573.82,694.62,597.17,479.65,300.08,100.23,0.00,0.00,201.26,302.49,404.53,425.60,324.92,244.42,163.43,81.96,20.51,20.51,0.00,0.00,465.66,799.19,1132.45,1384.57,1985.86,2037.81,1432.85,803.11,508.75,276.12,0.00,0.00,320.84,1069.47,427.79,150.02,0.00,302.45,628.39,606.72,108.34,0.00,326.98,350.18,131.71,22.00,0.00,641.71,1751.59,1067.45,267.40,0.00,0.00,649.44,1012.80,765.99,135.72,0.00,682.65,1507.84,596.37,184.05,0.00,0.00,0.00,116.42,116.77,175.32,176.20,235.41,236.58,296.91,297.80,358.79,360.58,421.10,422.37,485.12,487.54,551.23,552.88,1110.19,1111.30,1239.72,1245.91,1371.88,1375.99,1252.15,1257.16,1009.75,1009.75,761.10,761.86,508.42,509.43,255.99,256.25,231.08,231.78,206.85,206.85,181.53,181.90,181.90,181.90,0.00,52.18,52.39,130.97,131.36,210.18,210.18,289.57,290.15,371.13,372.24,452.46,452.92,534.97,537.11,620.15,622.01,706.66,710.19,796.10,797.69,881.97,881.97,969.48,972.39,1056.79,1061.02,838.48,838.48,784.15,787.29,677.52,680.90,568.55,568.55,426.84,426.84,285.13,286.27,143.71,144.28,0.00,289.72,290.01,291.17,291.75,437.63,439.82,442.02,442.90,445.12,446.90,597.05,598.84,660.05,661.37,634.46,637.00,609.09,610.31,489.22,491.18,430.64,430.64,369.49,370.60,310.07,311.31,250.29,251.29,189.04,189.79,126.78,126.78,63.58,63.64,31.98,32.05,32.11,32.24,32.33,32.50,0.00,553.54,554.09,737.02,740.71,1091.81,1095.08,1283.98,1287.83,1474.75,1476.23,1834.00,1834.00,2094.73,2094.73,2253.50,2253.50,3086.83,3099.17,3241.87,3258.08,3477.71,3477.71,3356.68,3373.46,3069.64,3069.64,2383.86,2383.86,1898.74,1900.64,1338.83,1341.50,1097.67,1103.16,854.91,858.33,646.32,647.61,468.66,470.06,0.00,545.10,545.10,637.85,641.04,916.69,920.36,1017.45,1022.54,1115.50,1118.85,1404.15,1411.17,1582.09,1588.42,1648.44,1651.74,2468.12,2480.46,2521.14,2521.14,2635.73,2646.28,2384.95,2392.10,1938.76,1944.58,1485.27,1488.24,1021.32,1025.41,553.25,554.35,437.74,439.05,359.59,360.66,320.91,321.23,321.23,322.84,0.00,486.68,488.14,610.79,612.62,943.11,944.05,1072.52,1076.81,1204.66,1205.87,1538.52,1543.14,1758.67,1763.95,1853.49,1857.20,1941.62,1945.50,1910.82,1912.73,1959.15,1959.15,2005.74,2013.76,2060.72,2071.03,1646.12,1646.12,1477.26,1477.26,1216.57,1216.57,960.66,964.50,701.45,702.86,483.21,484.66,264.63,265.42,0.00,312.76,312.76,560.74,561.30,812.31,815.56,1070.08,1075.43,1331.11,1336.43,1593.07,1599.44,1859.99,1869.29,2136.95,2143.36,3344.59,3344.59,3728.88,3743.80,4127.04,4143.55,4056.39,4064.51,0.00,0.00,0.00,0.00,0.00]

# NOTE: You cannot spend more than $2000 per a channel since that is all the modeling data has modeled.  If you do, you will see negative returns.

#### Scenario 1 - We spend $2000 on each campaign individually (run each campaign individually)

In [ ]:
# Scenario 1 - We spend $2000 on each campaign individually (run each campaign individually)

# Scenarios
# --> 1. We spend $2000 on each campaign individually (run each campaign individually)
# 2. We spend $3000 on autonomous_taxi and youtube_shorts individually
# 3. We spend $2000 on all 3 campaigns at the same time

# We will run #1, but will run each campaign individually. We will have 3 array (a, b, c) = (autonomous_taxi, youtube_short, google_mobile_ads)
scenario_1_a_autonomous_taxi_cumulative_spend = autonomous_taxi_cumulative_spend.copy()
scenario_1_a_autonomous_taxi_campaign_day = autonomous_taxi_campaign_day.copy()
scenario_1_a_youtube_short_cumulative_spend = youtube_short_cumulative_spend.copy()
scenario_1_a_youtube_short_campaign_day = youtube_short_campaign_day.copy()
scenario_1_a_google_mobile_ads_cumulative_spend = google_mobile_ads_cumulative_spend.copy()
scenario_1_a_google_mobile_ads_campaign_day = google_mobile_ads_campaign_day.copy()
scenario_1_a_sales_increase = sales_increase.copy()

scenario_1_b_autonomous_taxi_cumulative_spend = autonomous_taxi_cumulative_spend.copy()
scenario_1_b_autonomous_taxi_campaign_day = autonomous_taxi_campaign_day.copy()
scenario_1_b_youtube_short_cumulative_spend = youtube_short_cumulative_spend.copy()
scenario_1_b_youtube_short_campaign_day = youtube_short_campaign_day.copy()
scenario_1_b_google_mobile_ads_cumulative_spend = google_mobile_ads_cumulative_spend.copy()
scenario_1_b_google_mobile_ads_campaign_day = google_mobile_ads_campaign_day.copy()
scenario_1_b_sales_increase = sales_increase.copy()

scenario_1_c_autonomous_taxi_cumulative_spend = autonomous_taxi_cumulative_spend.copy()
scenario_1_c_autonomous_taxi_campaign_day = autonomous_taxi_campaign_day.copy()
scenario_1_c_youtube_short_cumulative_spend = youtube_short_cumulative_spend.copy()
scenario_1_c_youtube_short_campaign_day = youtube_short_campaign_day.copy()
scenario_1_c_google_mobile_ads_cumulative_spend = google_mobile_ads_cumulative_spend.copy()
scenario_1_c_google_mobile_ads_campaign_day = google_mobile_ads_campaign_day.copy()
scenario_1_c_sales_increase = sales_increase.copy()

# We will spend $2000 on each so we will use $125 a day which is 2000/125 = 16 (so we need to add 16 array elements to our arrays)
cumulative_amount = 0
day = 0
for i in range(16):
  cumulative_amount += 125
  day = day + 1
  scenario_1_a_autonomous_taxi_cumulative_spend.append(cumulative_amount)
  scenario_1_a_autonomous_taxi_campaign_day.append(str(day))
  scenario_1_a_youtube_short_cumulative_spend.append(0)
  scenario_1_a_youtube_short_campaign_day.append("")
  scenario_1_a_google_mobile_ads_cumulative_spend.append(0)
  scenario_1_a_google_mobile_ads_campaign_day.append("")

  scenario_1_b_autonomous_taxi_cumulative_spend.append(0)
  scenario_1_b_autonomous_taxi_campaign_day.append("")
  scenario_1_b_youtube_short_cumulative_spend.append(cumulative_amount)
  scenario_1_b_youtube_short_campaign_day.append(str(day))
  scenario_1_b_google_mobile_ads_cumulative_spend.append(0)
  scenario_1_b_google_mobile_ads_campaign_day.append("")

  scenario_1_c_autonomous_taxi_cumulative_spend.append(0)
  scenario_1_c_autonomous_taxi_campaign_day.append("")
  scenario_1_c_youtube_short_cumulative_spend.append(0)
  scenario_1_c_youtube_short_campaign_day.append("")
  scenario_1_c_google_mobile_ads_cumulative_spend.append(cumulative_amount)
  scenario_1_c_google_mobile_ads_campaign_day.append(str(day))


In [ ]:
# This is our sales increase data that we want to predict
inputs = [scenario_1_a_sales_increase, scenario_1_b_sales_increase, scenario_1_c_sales_increase]

# These are our categorical covariates (additional factors that we think might influence the thing we're trying to predict).
# Here we consider the day of the campaign, how long has it been running, we do not want the model to treat this as an integer so we use a categorical covariate
dynamic_categorical_covariates = {
    "autonomous_taxi_campaign_day" : [scenario_1_a_autonomous_taxi_campaign_day, scenario_1_b_autonomous_taxi_campaign_day, scenario_1_c_autonomous_taxi_campaign_day],
    "youtube_short_campaign_day" : [scenario_1_a_youtube_short_campaign_day, scenario_1_b_youtube_short_campaign_day ,scenario_1_c_youtube_short_campaign_day],
    "google_mobile_ads_campaign_day" : [scenario_1_a_google_mobile_ads_campaign_day, scenario_1_b_google_mobile_ads_campaign_day ,scenario_1_c_google_mobile_ads_campaign_day],
}

# These are our numerical covariates (additional numeric factors, just like the categories, but numbers)
# Here we consider the budget (cummlative spend) of our campaigns
dynamic_numerical_covariates = {
    "autonomous_taxi_cumulative_spend" : [scenario_1_a_autonomous_taxi_cumulative_spend, scenario_1_b_autonomous_taxi_cumulative_spend, scenario_1_c_autonomous_taxi_cumulative_spend],
    "youtube_short_cumulative_spend" : [scenario_1_a_youtube_short_cumulative_spend, scenario_1_b_youtube_short_cumulative_spend ,scenario_1_c_youtube_short_cumulative_spend],
    "google_mobile_ads_cumulative_spend" : [scenario_1_a_google_mobile_ads_cumulative_spend, scenario_1_b_google_mobile_ads_cumulative_spend ,scenario_1_c_google_mobile_ads_cumulative_spend],
}

# These are our static covariates (additional factors that we think are fixed)
static_numerical_covariates = {
}

# These are our static categorical covariates (additional factors that we think are fixed)
static_categorical_covariates = {
}

# frequency of each context time series. 0 for high frequency (default), 1 for medium, and 2 for low.
frequency = [0, 0, 0]

# scenario_1_model_forecast, scenario_1_xreg_forecast = model.forecast_with_covariates(
#     inputs=inputs,
#     dynamic_categorical_covariates=dynamic_categorical_covariates,
#     dynamic_numerical_covariates=dynamic_numerical_covariates,
#     static_numerical_covariates=static_numerical_covariates,
#     static_categorical_covariates=static_categorical_covariates,
#     freq=frequency,
#     xreg_mode="xreg + timesfm",              # default
#     ridge=0.0,
#     force_on_cpu=False,
#     normalize_xreg_target_per_input=True,    # default
# )

In [ ]:
# Create the JSON payload to the Vertex Model Endpoint
# You can have multiple array elements under "instances" and TimesFM will create a prediction for each
# The array elements from above are placed in the payload
payload = {
  "instances": [
    {
        "input": inputs[0],
        "freq": frequency[0],
        "horizon": horizon_len,
        "dynamic_numerical_covariates": {
            "autonomous_taxi_cumulative_spend": dynamic_numerical_covariates["autonomous_taxi_cumulative_spend"][0],
            "youtube_short_cumulative_spend": dynamic_numerical_covariates["youtube_short_cumulative_spend"][0],
            "google_mobile_ads_cumulative_spend": dynamic_numerical_covariates["google_mobile_ads_cumulative_spend"][0],
        },
        "dynamic_categorical_covariates": {
            "autonomous_taxi_campaign_day": dynamic_categorical_covariates["autonomous_taxi_campaign_day"][0],
            "youtube_short_campaign_day": dynamic_categorical_covariates["youtube_short_campaign_day"][0],
            "google_mobile_ads_campaign_day": dynamic_categorical_covariates["google_mobile_ads_campaign_day"][0],
        },
        "static_numerical_covariates": {
        },
        "static_categorical_covariates": {
        },
        "xreg_kwargs": {
            "xreg_mode" : xreg_mode
        }
    },
    {
        "input": inputs[1],
        "freq": frequency[1],
        "horizon": horizon_len,
        "dynamic_numerical_covariates": {
            "autonomous_taxi_cumulative_spend": dynamic_numerical_covariates["autonomous_taxi_cumulative_spend"][1],
            "youtube_short_cumulative_spend": dynamic_numerical_covariates["youtube_short_cumulative_spend"][1],
            "google_mobile_ads_cumulative_spend": dynamic_numerical_covariates["google_mobile_ads_cumulative_spend"][1],
        },
        "dynamic_categorical_covariates": {
            "autonomous_taxi_campaign_day": dynamic_categorical_covariates["autonomous_taxi_campaign_day"][1],
            "youtube_short_campaign_day": dynamic_categorical_covariates["youtube_short_campaign_day"][1],
            "google_mobile_ads_campaign_day": dynamic_categorical_covariates["google_mobile_ads_campaign_day"][1],
        },
        "static_numerical_covariates": {
        },
        "static_categorical_covariates": {
        },
        "xreg_kwargs": {
            "xreg_mode" : xreg_mode
        }
    },
    {
        "input": inputs[2],
        "freq": frequency[2],
        "horizon": horizon_len,
        "dynamic_numerical_covariates": {
            "autonomous_taxi_cumulative_spend": dynamic_numerical_covariates["autonomous_taxi_cumulative_spend"][2],
            "youtube_short_cumulative_spend": dynamic_numerical_covariates["youtube_short_cumulative_spend"][2],
            "google_mobile_ads_cumulative_spend": dynamic_numerical_covariates["google_mobile_ads_cumulative_spend"][2],
        },
        "dynamic_categorical_covariates": {
            "autonomous_taxi_campaign_day": dynamic_categorical_covariates["autonomous_taxi_campaign_day"][2],
            "youtube_short_campaign_day": dynamic_categorical_covariates["youtube_short_campaign_day"][2],
            "google_mobile_ads_campaign_day": dynamic_categorical_covariates["google_mobile_ads_campaign_day"][2],
        },
        "static_numerical_covariates": {
        },
        "static_categorical_covariates": {
        },
        "xreg_kwargs": {
            "xreg_mode" : xreg_mode
        }
    },

  ]
}

In [ ]:
# Get the project number in order to call teh endpoint
project_number = getProjectNumber(project_id)

In [ ]:
# Calls TimeFM to make a prediction
times_fm_inference = timesFMInference(project_number, endpoint_id, payload)

In [ ]:
# Create an array of forecasted elements
scenario_1_model_forecast = [times_fm_inference["predictions"][0]["point_forecast"],
                            times_fm_inference["predictions"][1]["point_forecast"],
                            times_fm_inference["predictions"][2]["point_forecast"]]

In [ ]:
# Show the predicted values for our campaign, notice how the amounts increase then sort of level off or decrease

print("$2000 on autonomous_taxi:", scenario_1_model_forecast[0])
print()
print("$2000 on youtube_short:", scenario_1_model_forecast[1])
print()
print("$2000 on google_mobile_ads:", scenario_1_model_forecast[2])
print()

In [ ]:
scenario_1_sum_forecast_autonomous_taxi = sum(scenario_1_model_forecast[0])
scenario_1_sum_forecast_youtube_short = sum(scenario_1_model_forecast[1])
scenario_1_sum_forecast_google_mobile_ads = sum(scenario_1_model_forecast[2])

print("Sum of forecast_autonomous_taxi:", int(scenario_1_sum_forecast_autonomous_taxi))
print("Sum of forecast_youtube_short:", int(scenario_1_sum_forecast_youtube_short))
print("Sum of forecast_google_mobile_ads:", int(scenario_1_sum_forecast_google_mobile_ads))

print()
print("Profit of forecast_autonomous_taxi:", int(scenario_1_sum_forecast_autonomous_taxi - 2000))
print("Profit of forecast_youtube_short:", int(scenario_1_sum_forecast_youtube_short - 2000))
print("Profit of forecast_google_mobile_ads:", int(scenario_1_sum_forecast_google_mobile_ads - 2000))

print()
print("Total Profit: ", int(scenario_1_sum_forecast_autonomous_taxi + scenario_1_sum_forecast_youtube_short + scenario_1_sum_forecast_google_mobile_ads - 6000))

In [ ]:
import matplotlib.pyplot as plt

# Subtract out the cost of the campaign $125 per day
scenario_1_forecast_autonomous_taxi = []
for item in scenario_1_model_forecast[0]:
  scenario_1_forecast_autonomous_taxi.append(item - 125)

scenario_1_forecast_youtube_short = []
for item in scenario_1_model_forecast[1]:
  scenario_1_forecast_youtube_short.append(item - 125)

scenario_1_forecast_google_mobile_ads = []
for item in scenario_1_model_forecast[2]:
  scenario_1_forecast_google_mobile_ads.append(item - 125)

scenario_1_timeline = list(range(1, 17))

plt.figure(figsize=(14, 6))

# forecast_autonomous_taxi
plt.plot(scenario_1_timeline, scenario_1_forecast_autonomous_taxi, label='Autonomous Taxi', marker='s', linestyle='-.', color='#{:02x}{:02x}{:02x}'.format(92, 200, 243))

# forecast_youtube_short
plt.plot(scenario_1_timeline, scenario_1_forecast_youtube_short, label='Youtube Short', marker='^', linestyle='--', color='#{:02x}{:02x}{:02x}'.format(53, 106, 228))

# forecast_google_mobile_ads
plt.plot(scenario_1_timeline, scenario_1_forecast_google_mobile_ads, label='Google Mobile Ads', marker='x', linestyle='--', color='#{:02x}{:02x}{:02x}'.format(144, 173, 59))

# Set labels and title
plt.xlabel('Campaign Day')
plt.ylabel('Predicted Sales Increase')
plt.title('Predicted Sales Increases (Profit) - $2000 on each Channel')

# Legend
plt.legend()

# Show all values
plt.xticks(scenario_1_timeline)

# Show the plot
plt.tight_layout()
plt.show()

#### Scenerio 2 - We spend $3000 on autonomous_taxi and youtube_shorts individually

In [ ]:
# Scenario 2 - We spend $3000 on autonomous_taxi and youtube_shorts individually

# Scenarios
# 1. We spend $2000 on each campaign individually (run each campaign individually)
# --> 2. We spend $3000 on autonomous_taxi and youtube_shorts individually
# 3. We spend $2000 on all 3 campaigns at the same time

# We will run #2, but will run each campaign individually. We will have 2 array (a, b) = (autonomous_taxi, youtube_short)
scenario_2_a_autonomous_taxi_cumulative_spend = autonomous_taxi_cumulative_spend.copy()
scenario_2_a_autonomous_taxi_campaign_day = autonomous_taxi_campaign_day.copy()
scenario_2_a_youtube_short_cumulative_spend = youtube_short_cumulative_spend.copy()
scenario_2_a_youtube_short_campaign_day = youtube_short_campaign_day.copy()
scenario_2_a_sales_increase = sales_increase.copy()

scenario_2_b_autonomous_taxi_cumulative_spend = autonomous_taxi_cumulative_spend.copy()
scenario_2_b_autonomous_taxi_campaign_day = autonomous_taxi_campaign_day.copy()
scenario_2_b_youtube_short_cumulative_spend = youtube_short_cumulative_spend.copy()
scenario_2_b_youtube_short_campaign_day = youtube_short_campaign_day.copy()
scenario_2_b_sales_increase = sales_increase.copy()

# We will spend $3000 on each so we will use $200 a day which is $3000/200 = 15 (so we need to add 10 array elements to our arrays)
cumulative_amount = 0
day = 0
for i in range(15):
  cumulative_amount += 200
  day = day + 1
  scenario_2_a_autonomous_taxi_cumulative_spend.append(cumulative_amount)
  scenario_2_a_autonomous_taxi_campaign_day.append(str(day))
  scenario_2_a_youtube_short_cumulative_spend.append(0)
  scenario_2_a_youtube_short_campaign_day.append("")

  scenario_2_b_autonomous_taxi_cumulative_spend.append(0)
  scenario_2_b_autonomous_taxi_campaign_day.append("")
  scenario_2_b_youtube_short_cumulative_spend.append(cumulative_amount)
  scenario_2_b_youtube_short_campaign_day.append(str(day))

In [ ]:
# This is our sales increase data that we want to predict
inputs = [scenario_2_a_sales_increase, scenario_2_b_sales_increase]

# These are our categorical covariates (additional factors that we think might influence the thing we're trying to predict).
# Here we consider the day of the campaign, how long has it been running, we do not want the model to treat this as an integer so we use a categorical covariate
dynamic_categorical_covariates = {
    "autonomous_taxi_campaign_day" : [scenario_2_a_autonomous_taxi_campaign_day, scenario_2_b_autonomous_taxi_campaign_day],
    "youtube_short_campaign_day" : [scenario_2_a_youtube_short_campaign_day, scenario_2_b_youtube_short_campaign_day],
}

# These are our numerical covariates (additional numeric factors, just like the categories, but numbers)
# Here we consider the budget (cummlative spend) of our campaigns
dynamic_numerical_covariates = {
    "autonomous_taxi_cumulative_spend" : [scenario_2_a_autonomous_taxi_cumulative_spend, scenario_2_b_autonomous_taxi_cumulative_spend],
    "youtube_short_cumulative_spend" : [scenario_2_a_youtube_short_cumulative_spend, scenario_2_b_youtube_short_cumulative_spend],
}

# These are our static covariates (additional factors that we think are fixed)
static_numerical_covariates = {
}

# These are our static categorical covariates (additional factors that we think are fixed)
static_categorical_covariates = {
}

# frequency of each context time series. 0 for high frequency (default), 1 for medium, and 2 for low.
frequency = [0, 0]

# scenario_2_model_forecast, scenario_2_xreg_forecast = model.forecast_with_covariates(
#     inputs=inputs,
#     dynamic_categorical_covariates=dynamic_categorical_covariates,
#     dynamic_numerical_covariates=dynamic_numerical_covariates,
#     static_numerical_covariates=static_numerical_covariates,
#     static_categorical_covariates=static_categorical_covariates,
#     freq=frequency,
#     xreg_mode="xreg + timesfm",              # default
#     ridge=0.0,
#     force_on_cpu=False,
#     normalize_xreg_target_per_input=True,    # default
# )

In [ ]:
# Create the JSON payload to the Vertex Model Endpoint
# You can have multiple array elements under "instances" and TimesFM will create a prediction for each
# The array elements from above are placed in the payload
payload = {
  "instances": [
    {
        "input": inputs[0],
        "freq": frequency[0],
        "horizon": horizon_len,
        "dynamic_numerical_covariates": {
            "autonomous_taxi_cumulative_spend": dynamic_numerical_covariates["autonomous_taxi_cumulative_spend"][0],
            "youtube_short_cumulative_spend": dynamic_numerical_covariates["youtube_short_cumulative_spend"][0]
        },
        "dynamic_categorical_covariates": {
            "autonomous_taxi_campaign_day": dynamic_categorical_covariates["autonomous_taxi_campaign_day"][0],
            "youtube_short_campaign_day": dynamic_categorical_covariates["youtube_short_campaign_day"][0]
        },
        "static_numerical_covariates": {
        },
        "static_categorical_covariates": {
        },
        "xreg_kwargs": {
            "xreg_mode" : xreg_mode
        }
    },
    {
        "input": inputs[1],
        "freq": frequency[1],
        "horizon": horizon_len,
        "dynamic_numerical_covariates": {
            "autonomous_taxi_cumulative_spend": dynamic_numerical_covariates["autonomous_taxi_cumulative_spend"][1],
            "youtube_short_cumulative_spend": dynamic_numerical_covariates["youtube_short_cumulative_spend"][1]
        },
        "dynamic_categorical_covariates": {
            "autonomous_taxi_campaign_day": dynamic_categorical_covariates["autonomous_taxi_campaign_day"][1],
            "youtube_short_campaign_day": dynamic_categorical_covariates["youtube_short_campaign_day"][1]
        },
        "static_numerical_covariates": {
        },
        "static_categorical_covariates": {
        },
        "xreg_kwargs": {
            "xreg_mode" : xreg_mode
        }
    }
  ]
}

In [ ]:
# Calls TimeFM to make a prediction
times_fm_inference = timesFMInference(project_number, endpoint_id, payload)

In [ ]:
# Create an array of forecasted elements
scenario_2_model_forecast = [times_fm_inference["predictions"][0]["point_forecast"],
                            times_fm_inference["predictions"][1]["point_forecast"]]

In [ ]:
# Show the predicted values for our campaign, notice how the amounts increase then sort of level off or decrease

print("$3000 on autonomous_taxi:", scenario_2_model_forecast[0])
print()
print("$3000 on youtube_short:", scenario_2_model_forecast[1])
print()

In [ ]:
scenario_2_sum_forecast_autonomous_taxi = sum(scenario_2_model_forecast[0])
scenario_2_sum_forecast_youtube_short = sum(scenario_2_model_forecast[1])

print("Sum of forecast_autonomous_taxi:", int(scenario_2_sum_forecast_autonomous_taxi))
print("Sum of forecast_youtube_short:", int(scenario_2_sum_forecast_youtube_short))

print()
print("Profit of forecast_autonomous_taxi:", int(scenario_2_sum_forecast_autonomous_taxi - 3000))
print("Profit of forecast_youtube_short:", int(scenario_2_sum_forecast_youtube_short - 3000))

print()
print("Total Profit: ", int(scenario_2_sum_forecast_autonomous_taxi + scenario_2_sum_forecast_youtube_short - 6000))

In [ ]:
import matplotlib.pyplot as plt

# Subtract out the cost of the campaign $200 per day
scenario_2_forecast_autonomous_taxi = []
for item in scenario_2_model_forecast[0]:
  scenario_2_forecast_autonomous_taxi.append(item - 200)

scenario_2_forecast_youtube_short = []
for item in scenario_2_model_forecast[1]:
  scenario_2_forecast_youtube_short.append(item - 200)

scenario_2_timeline = list(range(1, 16))

plt.figure(figsize=(14, 6))

# forecast_autonomous_taxin
plt.plot(scenario_2_timeline, scenario_2_forecast_autonomous_taxi, label='Autonomous Taxi', marker='s', linestyle='-.', color='#{:02x}{:02x}{:02x}'.format(92, 200, 243))

# forecast_youtube_short
plt.plot(scenario_2_timeline, scenario_2_forecast_youtube_short, label='Youtube Short', marker='^', linestyle='--', color='#{:02x}{:02x}{:02x}'.format(53, 106, 228))

# Set labels and title
plt.xlabel('Campaign Day')
plt.ylabel('Predicted Sales Increase')
plt.title('Predicted Sales Increases (Profit) - $3000 on each Channel')

# Legend
plt.legend()

# Show all values
plt.xticks(scenario_2_timeline)

# Show the plot
plt.tight_layout()
plt.show()

#### Scenerio 3 - We spend $2000 on all 3 campaigns at the same time

In [ ]:
# Scenario 3 - We spend $2000 on all 3 campaigns at the same time

# Scenarios
# 1. We spend $2000 on each campaign individually (run each campaign individually)
# 2. We spend $3000 on autonomous_taxi and youtube_shorts individually
# --> 3. We spend $2000 on all 3 campaigns at the same time

# We will run #3, but will run all the campaigns at once. We will have 3 array (a) = (autonomous_taxi, youtube_short, google_mobile_ads)
scenario_3_a_autonomous_taxi_cumulative_spend = autonomous_taxi_cumulative_spend.copy()
scenario_3_a_autonomous_taxi_campaign_day = autonomous_taxi_campaign_day.copy()
scenario_3_a_youtube_short_cumulative_spend = youtube_short_cumulative_spend.copy()
scenario_3_a_youtube_short_campaign_day = youtube_short_campaign_day.copy()
scenario_3_a_google_mobile_ads_cumulative_spend = google_mobile_ads_cumulative_spend.copy()
scenario_3_a_google_mobile_ads_campaign_day = google_mobile_ads_campaign_day.copy()
scenario_3_a_sales_increase = sales_increase.copy()

# We will spend $2000 on each so we will use $125 a day which is 2000/125 = 16 (so we need to add 16 array elements to our arrays)
cumulative_amount = 0
day = 0
for i in range(16):
  cumulative_amount += 125
  day = day + 1
  scenario_3_a_autonomous_taxi_cumulative_spend.append(cumulative_amount)
  scenario_3_a_autonomous_taxi_campaign_day.append(str(day))
  scenario_3_a_youtube_short_cumulative_spend.append(cumulative_amount)
  scenario_3_a_youtube_short_campaign_day.append(str(day))
  scenario_3_a_google_mobile_ads_cumulative_spend.append(cumulative_amount)
  scenario_3_a_google_mobile_ads_campaign_day.append(str(day))

In [ ]:
# This is our sales increase data that we want to predict
inputs = [scenario_3_a_sales_increase]

# These are our categorical covariates (additional factors that we think might influence the thing we're trying to predict).
# Here we consider the day of the campaign, how long has it been running, we do not want the model to treat this as an integer so we use a categorical covariate
dynamic_categorical_covariates = {
    "autonomous_taxi_campaign_day" : [scenario_3_a_autonomous_taxi_campaign_day],
    "youtube_short_campaign_day" : [scenario_3_a_youtube_short_campaign_day],
    "google_mobile_ads_campaign_day" : [scenario_3_a_google_mobile_ads_campaign_day],
}

# These are our numerical covariates (additional numeric factors, just like the categories, but numbers)
# Here we consider the budget (cummlative spend) of our campaigns
dynamic_numerical_covariates = {
    "autonomous_taxi_cumulative_spend" : [scenario_3_a_autonomous_taxi_cumulative_spend],
    "youtube_short_cumulative_spend" : [scenario_3_a_youtube_short_cumulative_spend],
    "google_mobile_ads_cumulative_spend" : [scenario_3_a_google_mobile_ads_cumulative_spend],
}

# These are our static covariates (additional factors that we think are fixed)
static_numerical_covariates = {
}

# These are our static categorical covariates (additional factors that we think are fixed)
static_categorical_covariates = {
}

# frequency of each context time series. 0 for high frequency (default), 1 for medium, and 2 for low.
frequency = [0]

# scenario_3_model_forecast, scenario_3_xreg_forecast = model.forecast_with_covariates(
#     inputs=inputs,
#     dynamic_categorical_covariates=dynamic_categorical_covariates,
#     dynamic_numerical_covariates=dynamic_numerical_covariates,
#     static_numerical_covariates=static_numerical_covariates,
#     static_categorical_covariates=static_categorical_covariates,
#     freq=frequency,
#     xreg_mode="xreg + timesfm",              # default
#     ridge=0.0,
#     force_on_cpu=False,
#     normalize_xreg_target_per_input=True,    # default
# )

In [ ]:
# Create the JSON payload to the Vertex Model Endpoint
# You can have multiple array elements under "instances" and TimesFM will create a prediction for each
# The array elements from above are placed in the payload
payload = {
  "instances": [
    {
        "input": inputs[0],
        "freq": frequency[0],
        "horizon": horizon_len,
        "dynamic_numerical_covariates": {
            "autonomous_taxi_cumulative_spend": dynamic_numerical_covariates["autonomous_taxi_cumulative_spend"][0],
            "youtube_short_cumulative_spend": dynamic_numerical_covariates["youtube_short_cumulative_spend"][0],
            "google_mobile_ads_cumulative_spend": dynamic_numerical_covariates["google_mobile_ads_cumulative_spend"][0],
        },
        "dynamic_categorical_covariates": {
            "autonomous_taxi_campaign_day": dynamic_categorical_covariates["autonomous_taxi_campaign_day"][0],
            "youtube_short_campaign_day": dynamic_categorical_covariates["youtube_short_campaign_day"][0],
            "google_mobile_ads_campaign_day": dynamic_categorical_covariates["google_mobile_ads_campaign_day"][0],
        },
        "static_numerical_covariates": {
        },
        "static_categorical_covariates": {
        },
        "xreg_kwargs": {
            "xreg_mode" : xreg_mode
        }
    }
  ]
}

In [ ]:
# Calls TimeFM to make a prediction
times_fm_inference = timesFMInference(project_number, endpoint_id, payload)

In [ ]:
# Create an array of forecasted elements
scenario_3_model_forecast = [times_fm_inference["predictions"][0]["point_forecast"]]

In [ ]:
# Show the predicted values for our campaign, notice how the amounts increase then sort of level off or decrease

print("$2000 on autonomous_taxi and youtube_short and google_mobile_ads:", scenario_3_model_forecast[0])

In [ ]:
scenario_3_sum_forecast_all = sum(scenario_3_model_forecast[0])

print("Sum of scenario_3_sum_forecast_all:", int(scenario_3_sum_forecast_all))

print()
print("Profit of forecast_autonomous_taxi:", int(scenario_3_sum_forecast_all - 6000))

print()
print("Total Profit: ", int(scenario_3_sum_forecast_all - 6000))

In [ ]:
import matplotlib.pyplot as plt

# Subtract out the cost of the campaign $125 per day
scenario_3_forecast_all = []
for item in scenario_3_model_forecast[0]:
  scenario_3_forecast_all.append(item - 125)

scenario_3_timeline = list(range(1, 17))

plt.figure(figsize=(14, 6))

# scenario_3_forecast_all
plt.plot(scenario_3_timeline, scenario_3_forecast_all, label='Autonomous Taxi', marker='s', linestyle='-.', color='#{:02x}{:02x}{:02x}'.format(92, 200, 243))

# Set labels and title
plt.xlabel('Campaign Day')
plt.ylabel('Predicted Sales Increase')
plt.title('Predicted Sales Increases (Profit) - $2000 on each Channel at the same time')

# Legend
plt.legend()

# Show all values
plt.xticks(scenario_3_timeline)

# Show the plot
plt.tight_layout()
plt.show()

### <font color='#4285f4'>Which Scenerio is the most Profitable?</font>

In [ ]:
import matplotlib.pyplot as plt

# Subtract out the cost of the campaign $6000
profits = [ int(scenario_1_sum_forecast_autonomous_taxi + scenario_1_sum_forecast_youtube_short + scenario_1_sum_forecast_google_mobile_ads - 6000) ,
            int(scenario_2_sum_forecast_autonomous_taxi + scenario_2_sum_forecast_youtube_short - 6000),
            int(scenario_3_sum_forecast_all - 6000)]

print(f"Profits: {profits}")

labels = ['Scenario 1', 'Scenario 2', 'Scenario 3']

plt.figure(figsize=(14, 6))
plt.bar(labels, profits, color=['skyblue', 'lightgreen', 'coral'], label=labels)

# Set labels and title
plt.xlabel('Scenario')
plt.ylabel('Predicted Sales Increase')
plt.title('Predicted Sales Increases (Profit) per Scenario')

# Legend
plt.legend()

# Show all values
plt.xticks()

# Show the plot
plt.tight_layout()
plt.show()

### <font color='#4285f4'>Clean Up</font>

**To save on costs:**

1. Open Vertex Model Registry
   - https://console.cloud.google.com/vertex-ai/models
2. Click on the model name
3. Under "Deploy your model" click the 3 dots and select "Undeploy Model"
4. Under "Deploy your model" click the 3 dots and select "Delete Endpoint"
5. Go back one screen and click the 3 dots and select "Delete Model"


### <font color='#4285f4'>Reference Links</font>


- [GitHub](https://github.com/google-research/timesfm)
- [Vertex AI Model Garden](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/timesfm)